# Exemplo simples de geração de tópicos com NMF

#### Vou tentar mostrar aqui como os tópicos são gerados a partir da matriz TFIDF para um caso bem simples

##### A parte abaixo pode ser ignorada, trata-se apenas de importaçoes que serão usadas

In [268]:
import warnings
warnings.filterwarnings('ignore')
from nltk.stem.snowball import PortugueseStemmer
from gensim.utils import simple_preprocess
import unicodedata
import gensim
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import itertools
from multiprocessing import Pool
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from leitor_discursos_camara_model import Camara_Sessao, Camara_FaseSessao, Camara_Discurso
import nomes_deputados
reload( nomes_deputados )
from nomes_deputados import lista_nomes_deputados, lista_partidos, vocab_especifico




### Aqui vou gerar um conjunto de textos simulando o que pode ocorrer na realidade do nosso conjunto  para responder a pergunta do André: Por exemplo, como os grupos temáticos são formados. Como os diversos grupos de orçamento foram formados, a partir do radical "orçament"

##### Vamos supor que temos 4 textos, pertencentes a 3 tópicos, todos com a palavra orcamento, um da saude e um sobre resposabilidade fiscal e outro sobre aumento de gastos para saude, e vamos supor ainda que a frequencia da palavra orcamento varie tambem de um texto para o outro.
##### Vou usar apenas palavras chaves para evitar fazer a limpeza dos textos

In [269]:
textos = [
    ['orcamento saude hospital medicos'], ## orçamento sobre saude
    ['orcamento loa responsabilidade qualidade orcamento lrf orcamento'], ## orçamento voltado para qualidade, responsabilidade
    ['orcamento hospital hospital saude precario orcamento'], ## orçamento sobre saúde - está precario!
    ['orcamento aumento gastos loa'] ## orçamento voltado para aumento de gastos
]
textos

[['orcamento saude hospital medicos'],
 ['orcamento loa responsabilidade qualidade orcamento lrf orcamento'],
 ['orcamento hospital hospital saude precario orcamento'],
 ['orcamento aumento gastos loa']]

In [270]:
df_textos = pd.DataFrame(textos, columns=['texto_inicial'])
df_textos

,texto_inicial
0,orcamento saude hospital medicos
1,orcamento loa responsabilidade qualidade orcam...
2,orcamento hospital hospital saude precario orc...
3,orcamento aumento gastos loa


#### Quebra em palavras:

In [271]:
df_textos['tokens'] = df_textos.texto_inicial.apply(simple_preprocess)
df_textos

,texto_inicial,tokens
0,orcamento saude hospital medicos,"[orcamento, saude, hospital, medicos]"
1,orcamento loa responsabilidade qualidade orcam...,"[orcamento, loa, qualidade, orcamento, lrf, or..."
2,orcamento hospital hospital saude precario orc...,"[orcamento, hospital, hospital, saude, precari..."
3,orcamento aumento gastos loa,"[orcamento, aumento, gastos, loa]"


#### Dicionario criado a partir dos textos

In [272]:
dicionario = gensim.corpora.Dictionary(df_textos.tokens)
dicionario_formato_nmf = {key: value for (value, key) in dicionario.iteritems()}
for k, v in sorted(dicionario.iteritems()):
    print k, v

0 saude
1 hospital
2 medicos
3 orcamento
4 loa
5 qualidade
6 lrf
7 precario
8 gastos
9 aumento


In [285]:
# Valores usados no modelo NMF
num_topicos = 3
n_top_words = 4

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topico #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [286]:
# tfidf para NMF.
tfidf_vectorizer = TfidfVectorizer(
                                   vocabulary=dicionario_formato_nmf
                                   )
tfidf = tfidf_vectorizer.fit_transform(df_textos.texto_inicial)

In [287]:
tfidf_coo = tfidf.tocoo(copy=False)

In [288]:
df_tfidf = pd.DataFrame({'documento': tfidf_coo.row, 'palavra': tfidf_coo.col, 'peso': tfidf_coo.data}
                 )[['documento', 'palavra', 'peso']].sort_values(['documento', 'palavra']
                 ).reset_index(drop=True)

### O resultado final da matriz TFIDF é dado abaixo, em uma representação simplificada (os valores zero não estão representados)

#### Os valores não tem uma interpretação direta, apenas guardam uma relação com a importância do termo para o texto, levando em conta, de forma inversa, a importância da palavra para todos os textos.

Só pra não ficar em branco de onde vem os numeros abaixo: Cada peso é calculado como:


\begin{equation*}
\text{tf-idf(t,d)}=\text{tf(t,d)} \times \text{idf(t)}.
\end{equation*}
Using the TfidfTransformer‘s default settings, the term frequency, tf(t,d), the number of times a term occurs in a given document, is multiplied with idf component, which is computed as
\begin{equation*}
\text{idf}(t) = log{\frac{1 + n_d}{1+\text{df}(d,t)}} + 1,
\end{equation*}
where n_d   is the total number of documents, and {df}(d,t) is the number of documents that contain term t. The resulting tf-idf vectors are then normalized by the Euclidean norm:
\begin{equation*}
v_{norm} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v{_1}^2 +
v{_2}^2 + \dots + v{_n}^2}}.
\end{equation*}

#### Por exemplo, no documento 0, apenas tem pesos associados as palavras 0,1,2,3, com maior peso para a palavra 2 (medicos)

In [289]:
df_tfidf

,documento,palavra,peso
0,0,0,0.497096
1,0,1,0.497096
2,0,2,0.630504
3,0,3,0.329023
4,1,3,0.695105
5,1,4,0.350061
6,1,5,0.444008
7,1,6,0.444008
8,2,0,0.345833
9,2,1,0.691666


### O objetivo do modelo NMF é gerar a decomposicao da matriz TFIDF.
A matriz é de dimensao Numero de documentos (4) x numero de termos (10) que poderiam estar presentes (total do dicionario)

In [290]:
tfidf

<4x10 sparse matrix of type '<type 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [291]:
# Modelo NMF 
nmf = NMF(n_components=3, random_state=1,
          alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

print("\nTopicos no NMF:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


Topicos no NMF:
Topico #0:
hospital saude orcamento medicos
Topico #1:
aumento gastos loa orcamento
Topico #2:
orcamento lrf qualidade loa
()


A decomposicao acima foi feita com o numero de tópicos = 3.  O modelo tenta minimizar o erro da decomposição da matriz tfidf em duas matrizes de componentes não negativos: 
#### tfidf(dimensao=4x10) = W(dimensao=4x3) *H(dimensao=3x10)


#### Desta forma, cada linha do componente W da decomposicao pode ser interpretado como um vetor que demonstra a presença de cada tópico nos textos:

In [281]:
ocorrencia_topicos =nmf.transform(tfidf)

for documento in ocorrencia_topicos:
    print list(documento)

[0.83455896264791718, 0.0, 0.0]
[0.0, 0.0, 1.0679449715896323]
[0.82643699146350214, 0.0, 0.024758729699659528]
[0.0, 1.0737573447675506, 0.0027315737106655036]


#### e cada linha do componente H demonstra a presenca de cada palavra nos tópicos:
 
 Observem a ordem das palavras no dicionario. Esta é a ordem das importancias associadas as palavras nos tópicos.

In [282]:
for topico in nmf.components_:
    print list(topico)
for k, v in sorted(dicionario.iteritems()):
    print k, v

[0.45519418881468132, 0.65514245604938082, 0.33313507265366404, 0.41344964696911196, 0.0, 0.0, 0.0, 0.21862793703546615, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.23082665467270774, 0.37145791192224714, 0.0, 0.0, 0.0, 0.48314310572128216, 0.48314310572128216]
[0.0, 0.0, 0.0, 0.58378844107696026, 0.27202869848146438, 0.35611145394712762, 0.35611145394712762, 0.0, 0.0, 0.0]
0 saude
1 hospital
2 medicos
3 orcamento
4 loa
5 qualidade
6 lrf
7 precario
8 gastos
9 aumento


### Assim, podemos voltar a pergunta do André:
#### Como os grupos temáticos são formados? 

Como tudo parte da matriz TFIDF, existem 3 componentes fundamentais para determinar os tópicos:
- A frequencia do termo no texto em relacao aos outros textos (vai determinar importancia da palavra para o texto)
- A proximidade desta importancia relativa na matriz TFIDF em textos diferentes. Ou seja, se uma palavra possui importancia relativa parecida em muitos textos, ela tem maior chance de aparecer de forma relevante dentro de um tópico.
- A co-ocorrencia de palavras nos textos, pois facilita a minimizacao do erro na estimativa da decomposição quando comparada a matriz TFIDF original.

Explorem o exemplo acima para entender melhor:
Por exemplo, para o tópico 0 (primeira linha), a maior importancia esta em hospital, depois saude, pois hospital ocorre mais vezes que saude, quando estes dois ocorrem juntos. Orcamento só é a terceira mais importante neste tópico, pois sua importancia na matriz é baixa, pois ocorre em todos os textos. Sua importancia será maior no tópico em que ele ocorrerá mais vezes.

### Como os diversos grupos de orçamento foram formados, a partir do radical "orçament" ?

obs: Acho que esqueci de falar sobre isso no outro email, mas a coluna ASSUNTO da planilha excel foi determinada por mim a partir das palavras e dos discursos, apos uma rápida observação. Talvez isso tenha gerado confusão.

Então da mesma forma que expliquei acima, a partir da quantidade de tópicos pré definida, o modelo se ajustou para encontrar palavras relevantes e que ocorrem de forma conjunta nos textos. Assim foram identificados alguns tópicos com relacionados a orçamento (na minha breve análise).




####  Sendo assim, podemos associar o tópico mais presente para cada discurso. Foi isso que fiz para analisar a evolucao no email anterior.

In [283]:
ocorrencia_topicos =nmf.transform(tfidf)

lista_topico_principal_por_discurso = []
for discurso in ocorrencia_topicos:
    topico, relevancia =  max(enumerate(discurso), key=lambda x: x[1])
    lista_topico_principal_por_discurso.append(topico)
df_textos['topico_principal'] = pd.Series(lista_topico_principal_por_discurso, index=df_textos.index)

In [284]:
df_textos

,texto_inicial,tokens,topico_principal
0,orcamento saude hospital medicos,"[orcamento, saude, hospital, medicos]",0
1,orcamento loa responsabilidade qualidade orcam...,"[orcamento, loa, qualidade, orcamento, lrf, or...",2
2,orcamento hospital hospital saude precario orc...,"[orcamento, hospital, hospital, saude, precari...",0
3,orcamento aumento gastos loa,"[orcamento, aumento, gastos, loa]",1
